In [1]:
#FilrougeML IO Librairy
# Dataframe <-> Json
# Structure du DataFrame
#   TitLe: titre de l'article STRING
#   Abstract: Résumé STRING
#   Body: contenu (corps) de l'article STRING
#   RefTo: Liste des DOI auquel se réfere l'article https://doi.org/leDOI
#   Authors DICTIONNAIRE des auteurs clef: Name, Id, Organisation
#   RefBy: List des DOI  des articles se référant à cet article
#   DatePub: Date de publication datetime64
#   url: URL de l'article: STRING
#   LibName: Nom de la bibliothéque ( site web) ou est stocké l'article STRING
import pandas as pd

In [102]:
gorient = 'columns'


def AddArticles(DOI, df=None, Title='', Abstract='', Body='', RefTo='', Authors='', RefBy='', DatePub='', url='', LibName=''):
    """Ajoute un article dans un dataframe typé et compatible pour sauvegarde
    Exemple d'utilisation:
    Premier appel de dataframe n'existe pas
    art = AddArticles('10.1128/MCB.22.9.2918-2927.2002',None,'Titre','abstract','body',[['10.1128/MCB.22.9.2918-2927.2002']],{'Name':'nomaut1','Id':'idaut1','Organization':'orgaut1'},[['10.1128/MCB.22.9.2918-2927.2002']],'2020 Feb 20','url','bib')
    Appel suivants, le datframe existe et est passé en parametre
    art = AddArticles('10.1rrr8/MCB.22.9.2918-2927.2002',art,'Titre2','abstract2','body2',[['22210.1128/MCB.22.9.2918-2927.2002']],{'Name':'nomaut2','Id2':'idaut2','Organization2':'orgaut21'},[['12220.1128/MCB.22.9.2918-2927.2002']],'2022 Feb 20','url2','bib2')
    """
    columns = ['DOI', 'Title', 'Abstract', 'Body', 'RefTo',
               'Authors', 'RefBy', 'DatePub', 'url', 'LibName']
    if isinstance(df, type(None)):
        df = pd.DataFrame([[DOI, Title, Abstract, Body, RefTo, Authors,
                          RefBy, DatePub, url, LibName]], columns = columns)
        df= df.astype({'DOI':'string','Title':'string','Abstract':'string','Body':'string','DatePub':'datetime64[ns]','url':'string','LibName':'string'})
    else:
        df = df.append(pd.DataFrame(
            [[DOI, Title, Abstract, Body, RefTo, Authors, RefBy, DatePub, url, LibName]], columns = columns), ignore_index=True)
    return(df)


In [103]:
art = AddArticles('10.1128/MCB.22.9.2918-2927.2002',None,'Titre','abstract','body',[['10.1128/MCB.22.9.2918-2927.2002']],{'Name':'nomaut1','Id':'idaut1','Organization':'orgaut1'},[['10.1128/MCB.22.9.2918-2927.2002']],'2020 Feb 20','url','bib')
art = AddArticles('10.1rrr8/MCB.22.9.2918-2927.2002',art,'Titre2','abstract2','body2',[['22210.1128/MCB.22.9.2918-2927.2002']],{'Name':'nomaut2','Id2':'idaut2','Organization2':'orgaut21'},[['12220.1128/MCB.22.9.2918-2927.2002']],'2022 Feb 20','url2','bib2')

In [104]:
##art= art.astype({'DOI':'string','Title':'string','Abstract':'string','Body':'string','DatePub':'datetime64[ns]','url':'string','LibName':'string'})
art

,DOI,Title,Abstract,Body,RefTo,Authors,RefBy,DatePub,url,LibName
0,10.1128/MCB.22.9.2918-2927.2002,Titre,abstract,body,[[10.1128/MCB.22.9.2918-2927.2002]],"{'Name': 'nomaut1', 'Id': 'idaut1', 'Organizat...",[[10.1128/MCB.22.9.2918-2927.2002]],2020-02-20 00:00:00,url,bib
1,10.1rrr8/MCB.22.9.2918-2927.2002,Titre2,abstract2,body2,[[22210.1128/MCB.22.9.2918-2927.2002]],"{'Name': 'nomaut2', 'Id2': 'idaut2', 'Organiza...",[[12220.1128/MCB.22.9.2918-2927.2002]],2022 Feb 20,url2,bib2


In [109]:
artjson = art.to_dict(orient='records')
artjson

[{'DOI': '10.1128/MCB.22.9.2918-2927.2002',
  'Title': 'Titre',
  'Abstract': 'abstract',
  'Body': 'body',
  'RefTo': [['10.1128/MCB.22.9.2918-2927.2002']],
  'Authors': {'Name': 'nomaut1', 'Id': 'idaut1', 'Organization': 'orgaut1'},
  'RefBy': [['10.1128/MCB.22.9.2918-2927.2002']],
  'DatePub': Timestamp('2020-02-20 00:00:00'),
  'url': 'url',
  'LibName': 'bib'},
 {'DOI': '10.1rrr8/MCB.22.9.2918-2927.2002',
  'Title': 'Titre2',
  'Abstract': 'abstract2',
  'Body': 'body2',
  'RefTo': [['22210.1128/MCB.22.9.2918-2927.2002']],
  'Authors': {'Name': 'nomaut2', 'Id2': 'idaut2', 'Organization2': 'orgaut21'},
  'RefBy': [['12220.1128/MCB.22.9.2918-2927.2002']],
  'DatePub': '2022 Feb 20',
  'url': 'url2',
  'LibName': 'bib2'}]

In [9]:
import configparser

config = configparser.ConfigParser()
config.read('elastic/credentials.ini')

['elastic/credentials.ini']

In [11]:
def get_database():
  import pymongo

  CONNECTION_STRING = f'mongodb+srv://{config["MONGODB"]["user"]}:{config["MONGODB"]["password"]}@{config["MONGODB"]["cluster"]}'
  # Create a connection using MongoClient. You can import MongoClient or use pymongo.MongoClient
  client = pymongo.MongoClient(CONNECTION_STRING)

  # Create the database for our example (we will use the same database throughout the tutorial
  return client[config["MONGODB"]["database"]]

In [110]:
dbname = get_database()
collection_name = dbname[config['MONGODB']['collection']]
collection_name.insert_many(artjson)

In [111]:
import pymongo
client = pymongo.MongoClient(f'mongodb://{config["DATALAKE"]["user"]}:{config["DATALAKE"]["password"]}@{config["DATALAKE"]["cluster"]}/{config["DATALAKE"]["database"]}?ssl=true&authSource=admin')
db = client[config["DATALAKE"]["database"]]
db

Database(MongoClient(host=['filrougeml22datalake-wlaxm.a.query.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', tls=True), 'pubmedDL')

In [134]:
test = db[config["DATALAKE"]["collection"]].find({ '$text': { '$search': "\"Titre\"" } }).limit(5)

for x in test:
  print(x)

OperationFailure: text index required for $text query, correlationID = 16d2c4d9c1cbf442f018449c, full error: {'ok': 0, 'errmsg': 'text index required for $text query, correlationID = 16d2c4d9c1cbf442f018449c', 'code': 27, 'codeName': 'IndexNotFound'}

In [143]:
test =db[config["DATALAKE"]["collection"]]

test

Collection(Database(MongoClient(host=['filrougeml22datalake-wlaxm.a.query.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', tls=True), 'pubmedDL'), 'articleDL')

In [144]:
test.create_index("Title", pymongo.TEXT)

AttributeError: 'str' object has no attribute 'in_transaction'